In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd drive/My Drive/AtoZ/Deep_Learning_A_Z/Boltzmann_Machines

[Errno 2] No such file or directory: 'drive/My Drive/AtoZ/Deep_Learning_A_Z/Boltzmann_Machines'
/content/drive/My Drive/AtoZ/Deep_Learning_A_Z/Boltzmann_Machines


In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [0]:
#Importing dataset
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding = 'latin-1')

In [0]:
#prepare training set and test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t', header=None)

test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t', header=None)

In [127]:
combined = pd.concat([training_set,test_set], axis=0)
nb_users = len(combined[0].unique()) #943
nb_movies = len(combined[1].unique()) #1682
training_set.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [182]:
training_set

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
5,1,7,4,875071561
6,1,8,1,875072484
7,1,9,5,878543541
8,1,11,2,875072262
9,1,13,5,875071805


In [0]:
x = training_set.copy()

In [0]:
def convert(data):
  new_data=[]
  for id_user in range(1, nb_users+1):
    user_data = x[x[0]==id_user]
    id_movies = user_data[1]
    id_ratings = user_data[2]
    ratings = np.zeros(nb_movies)
    ratings[id_movies-1]=id_ratings
    new_data.append(list(ratings))
  return new_data

In [0]:
ts = convert(x)

In [0]:
#manipulate
for id_user in range(0,2):
  user_ratings = ts[id_user]
  for id_movies in range(nb_movies):
    if user_ratings[id_movies]==0:
      user_ratings[id_movies]=-1
    if user_ratings[id_movies]==1 or user_ratings[id_movies]==2 :
      user_ratings[id_movies]=0
    if user_ratings[id_movies]>=3 :
      user_ratings[id_movies]=1

In [0]:
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh,nv) #Vector of weights
    self.a = torch.randn(1,nh) #bias of the hidden nodes
    self.b = torch.randn(1,nv) #bias of the visible nodes
  
  def sample_h(self,x): #x - visible neurons
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
    
  def sample_v(self,y): #y - hidden neurons
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(),ph0) - torch.mm(vk.t(),phk)).t()
    self.b += torch.sum((v0 - vk),0)
    self.a += torch.sum((ph0 - phk),0)

In [0]:
nh=100
nv=len(ts[0])
batch_size=100

In [0]:
#number of unique movies
vb = tf.placeholder(tf.float32,[nv])

#number if features that we are going to learn
hb = tf.placeholder(tf.float32,[nh])
W = tf.placeholder(tf.float32,[nv,nh])